#Predicting sentiment from product reviews

#Fire up GraphLab Create

In [2]:
import graphlab

In [3]:
import os
os.getcwd()

'/Users/AaronAn/MOOC/Machine Learning SP/1. ML Case Study/iPython Notebooks/Week 3'

#Read some product review data

Loading reviews for a set of baby products. 

In [4]:
products = graphlab.SFrame('/Users/AaronAn/MOOC/Machine Learning SP/1. ML Case Study/Lec Data/amazon_baby.gl/')

[INFO] This non-commercial license of GraphLab Create is assigned to ranan2014@u.northwestern.eduand will expire on September 25, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-511 - Server binary: /Users/AaronAn/.graphlab/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1444366746.log
[INFO] GraphLab Server Version: 1.6.1


#Let's explore this data together

Data includes the product name, the review text and the rating of the review. 

In [5]:
products.head()

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


You can specify how many rows to view in the () within head function. 

alt + Enter is incert below

#Build the word count vector for each review

adding a new column to the products called word count, the func is count_words

In [6]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

In [7]:
products.head()

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'and': 5, '6': 1,'stink': 1, 'because' ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3, 'love': 1,'it': 2, 'highly': 1, ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1, 'and':3, 'love': 2, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2, 'parents!!':1, 'all': 2, 'puppet.': ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2, 'this': 2,'her': 1, 'help': 2, ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1, 'noble': 1,'is': 1, 'it': 1, 'as': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2, 'all': 1,'right': 1, 'when': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1, 'help': 1,'give': 1, 'is': 1, ' ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1, 'nanny':1, 'standarad': 1, ..."


In [8]:
graphlab.canvas.set_target('ipynb')

In [9]:
products['name'].show()

#Examining the reviews for most-sold product:  'Vulli Sophie the Giraffe Teether'

In [10]:
giraffe_reviews = products[products['name'] == 'Vulli Sophie the Giraffe Teether']

this is the select in Sframe, similar to R. 

In [11]:
len(giraffe_reviews)

785

In [12]:
giraffe_reviews['rating'].show(view='Categorical')

#Build a sentiment classifier

In [13]:
products['rating'].show(view='Categorical')

##Define what's a positive and a negative sentiment

We will ignore all reviews with rating = 3, since they tend to have a neutral sentiment.  Reviews with a rating of 4 or higher will be considered positive, while the ones with rating of 2 or lower will have a negative sentiment.   

Subset the products, filter out all the products with a rating of 3. 

In [14]:
#ignore all 3* reviews
products = products[products['rating'] != 3]

In [15]:
#positive sentiment = 4* or 5* reviews
products['sentiment'] = products['rating'] >=4

# so this is a binary, not a boolean. 

In [16]:
products.head()

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3, 'love': 1,'it': 2, 'highly': 1, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1, 'and':3, 'love': 2, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2, 'parents!!':1, 'all': 2, 'puppet.': ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2, 'this': 2,'her': 1, 'help': 2, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1, 'noble': 1,'is': 1, 'it': 1, 'as': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2, 'all': 1,'right': 1, 'when': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1, 'help': 1,'give': 1, 'is': 1, ' ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1, 'nanny':1, 'standarad': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 1, 'forget': 1,'just': 1, 'food': 1, ...",1


##Let's train the sentiment classifier

In [17]:
train_data,test_data = products.random_split(.8, seed=0)

80% goes into the training dataset. We just use the word count as a feature, note here word count is a dictionary, to build the model. 

In [18]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=['word_count'],
                                                     validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 219217
PROGRESS: Number of coefficients    : 219218
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 5        | 0.000002  | 4.160651     | 0.841481          | 0.839989            |
PROGRESS: | 2         | 9        | 3.000000  | 6.514191     | 0.947425          | 0.894877            |
PROGRESS: | 3         | 10       | 3.000000  | 7.383439     | 0.92

#Evaluate the sentiment model

In [19]:
sentiment_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +------------------+----------------+------------------+-------+------+
 |    threshold     |      fpr       |       tpr        |   p   |  n   |
 +------------------+----------------+------------------+-------+------+
 |       0.0        | 0.217480816021 | 0.00596194352219 | 28011 | 5343 |
 | 0.0010000000475  | 0.782519183979 |  0.994038056478  | 28011 | 5343 |
 | 0.00200000009499 | 0.743028261276 |  0.992895648138  | 28011 | 5343 |
 | 0.00300000002608 | 0.721130451057 |  0.992074542144  | 28011 | 5343 |
 | 0.00400000018999 | 0.705596107056 |  0.991539038235  | 28011 | 5343 |
 | 0.00499999988824 | 0.69436646079  |  0.991182035629  | 28011 | 5343 |
 | 0.00600000005215 | 0.684821261464 |  0.99064653172   | 28011 | 5343 |
 | 0.00700000021607 | 0.674714579824 |  0.990218128592  | 28011 | 5343 |
 | 0.00800000037998 | 0.664607898185 |  0.989861125986  | 28011 | 5343 |
 | 0.00899999961257 

In [20]:
sentiment_model.show(view='Evaluation')

We can slide the point to change the cutting point. 

#Applying the learned model to understand sentiment for Giraffe

In [21]:
giraffe_reviews['predicted_sentiment'] = sentiment_model.predict(giraffe_reviews, output_type='probability')

You don't have to do the logit conversion, just specify the output_type = 'probability' is enough. 

In [22]:
giraffe_reviews.head()

name,review,rating,word_count,predicted_sentiment
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5.0,"{'and': 1, 'all': 1,'because': 1, 'it': 1, ...",0.999513023521
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5.0,"{'and': 1, 'right': 1,'help': 1, 'just': 1, ...",0.999320678306
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1.0,"{'and': 2, 'all': 1,'would': 1, 'latex.': 1, ...",0.013558811687
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5.0,"{'and': 2, 'one!': 1,'all': 1, 'love': 1, ...",0.995769474148
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5.0,"{'and': 3, 'all': 1,'months': 1, 'old': 1, ...",0.662374415673
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5.0,"{'and': 6, 'seven': 1,'already': 1, 'love': 1, ...",0.999997148186
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5.0,"{'and': 4, 'drooling,':1, 'love': 1, ...",0.989190989536
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5.0,"{'and': 3, 'chew': 1,'be': 1, 'is': 1, ...",0.999563518413
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5.0,"{'chew': 2, 'seemed': 1,'because': 1, 'about.': ...",0.970160542725
Vulli Sophie the GiraffeTeether ...,"My baby seems to likethis toy, but I could ...",3.0,"{'and': 2, 'already': 1,'some': 1, 'it': 3, ...",0.195367644588


##Sort the reviews based on the predicted sentiment and explore

In [23]:
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending=False)

In [24]:
giraffe_reviews.head()

name,review,rating,word_count,predicted_sentiment
Vulli Sophie the GiraffeTeether ...,"Sophie, oh Sophie, yourtime has come. My ...",5.0,"{'giggles': 1, 'all': 1,""violet's"": 2, 'bring': ...",1.0
Vulli Sophie the GiraffeTeether ...,I'm not sure why Sophieis such a hit with the ...,4.0,"{'adoring': 1, 'find': 1,'month': 1, 'bright': 1, ...",0.999999999703
Vulli Sophie the GiraffeTeether ...,I'll be honest...I boughtthis toy because all the ...,4.0,"{'all': 2, 'discovered':1, 'existence.': 1, ...",0.999999999392
Vulli Sophie the GiraffeTeether ...,We got this littlegiraffe as a gift from a ...,5.0,"{'all': 2, ""don't"": 1,'(literally).so': 1, ...",0.99999999919
Vulli Sophie the GiraffeTeether ...,As a mother of 16monthold twins; I bought ...,5.0,"{'cute': 1, 'all': 1,'reviews.': 2, 'just' ...",0.999999998657
Vulli Sophie the GiraffeTeether ...,Sophie the Giraffe is theperfect teething toy. ...,5.0,"{'just': 2, 'both': 1,'month': 1, 'ears,': 1, ...",0.999999997108
Vulli Sophie the GiraffeTeether ...,Sophie la giraffe isabsolutely the best toy ...,5.0,"{'and': 5, 'the': 1,'all': 1, 'that': 2, ...",0.999999995589
Vulli Sophie the GiraffeTeether ...,My 5-mos old son took tothis immediately. The ...,5.0,"{'just': 1, 'shape': 2,'mutt': 1, '""dog': 1, ...",0.999999995573
Vulli Sophie the GiraffeTeether ...,My nephews and my fourkids all had Sophie in ...,5.0,"{'and': 4, 'chew': 1,'all': 1, 'perfect;': 1, ...",0.999999989527
Vulli Sophie the GiraffeTeether ...,Never thought I'd see myson French kissing a ...,5.0,"{'giggles': 1, 'all': 1,'out,': 1, 'over': 1, ...",0.999999985069


##Most positive reviews for the giraffe

In [25]:
giraffe_reviews[0]['review']

"Sophie, oh Sophie, your time has come. My granddaughter, Violet is 5 months old and starting to teeth. What joy little Sophie brings to Violet. Sophie is made of a very pliable rubber that is sturdy but not tough. It is quite easy for Violet to twist Sophie into unheard of positions to get Sophie into her mouth. The little nose and hooves fit perfectly into small mouths, and the drooling has purpose. The paint on Sophie is food quality.Sophie was born in 1961 in France. The maker had wondered why there was nothing available for babies and made Sophie from the finest rubber, phthalate-free on St Sophie's Day, thus the name was born. Since that time millions of Sophie's populate the world. She is soft and for babies little hands easy to grasp. Violet especially loves the bumpy head and horns of Sophie. Sophie has a long neck that easy to grasp and twist. She has lovely, sizable spots that attract Violet's attention. Sophie has happy little squeaks that bring squeals of delight from Viol

In [26]:
giraffe_reviews[1]['review']

"I'm not sure why Sophie is such a hit with the little ones, but my 7 month old baby girl is one of her adoring fans.  The rubber is softer and more pleasant to handle, and my daughter has enjoyed chewing on her legs and the nubs on her head even before she started teething.  She also loves the squeak that Sophie makes when you squeeze her.  Not sure what it is but if Sophie is amongst a pile of her other toys, my daughter will more often than not reach for Sophie.  And I have the peace of mind of knowing that only edible and safe paints and materials have been used to make Sophie, as opposed to Bright Starts and other baby toys made in China.  Now that the research is out on phthalates and other toxic substances in baby toys, I think it's more important than ever to find good quality toys that are also safe for our babies to handle and put in their mouths.  Sophie is a must-have for every new mom in my opinion.  Even if your kid is one of the few that can take or leave her, it's worth

##Show most negative reviews for giraffe

Python access the end one. 

In [27]:
giraffe_reviews[-1]['review']

"My son (now 2.5) LOVED his Sophie, and I bought one for every baby shower I've gone to. Now, my daughter (6 months) just today nearly choked on it and I will never give it to her again. Had I not been within hearing range it could have been fatal. The strange sound she was making caught my attention and when I went to her and found the front curved leg shoved well down her throat and her face a purply/blue I panicked. I pulled it out and she vomited all over the carpet before screaming her head off. I can't believe how my opinion of this toy has changed from a must-have to a must-not-use. Please don't disregard any of the choking hazard comments, they are not over exaggerated!"

In [28]:
giraffe_reviews[-2]['review']

"This children's toy is nostalgic and very cute. However, there is a distinct rubber smell and a very odd taste, yes I tried it, that my baby did not enjoy. Also, if it is soiled it is extremely difficult to clean as the rubber is a kind of porus material and does not clean well. The final thing is the squeaking device inside which stopped working after the first couple of days. I returned this item feeling I had overpaid for a toy that was defective and did not meet my expectations. Please do not be swayed by the cute packaging and hype surounding it as I was. One more thing, I was given a full refund from Amazon without any problem."

## Quiz 3

In [29]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

#### 1. Use .apply() to build a new feature with the counts for each of the selected_words:

In [30]:
# create the awesome_count() function

def awesome_count(dict):
    if 'awesome' in dict:
        return dict['awesome']
    else:
        return 0

In [31]:
products['awesome'] = products['word_count'].apply(awesome_count)

In [32]:
def great_count(dict):
    if 'great' in dict:
        return dict['great']
    else:
        return 0

In [33]:
products['great'] = products['word_count'].apply(great_count)

In [34]:
def fantastic_count(dict):
    if 'fantastic' in dict:
        return dict['fantastic']
    else:
        return 0

In [35]:
products['fantastic'] = products['word_count'].apply(fantastic_count)

In [36]:
def amazing_count(dict):
    if 'amazing' in dict:
        return dict['amazing']
    else:
        return 0

In [37]:
products['amazing'] = products['word_count'].apply(amazing_count)

In [38]:
def love_count(dict):
    if 'love' in dict:
        return dict['love']
    else:
        return 0

In [39]:
products['love'] = products['word_count'].apply(love_count)

In [40]:
def horrible_count(dict):
    if 'horrible' in dict:
        return dict['horrible']
    else:
        return 0

In [41]:
products['horrible'] = products['word_count'].apply(horrible_count)

In [42]:
def bad_count(dict):
    if 'bad' in dict:
        return dict['bad']
    else:
        return 0

In [43]:
products['bad'] = products['word_count'].apply(bad_count)

In [44]:
def terrible_count(dict):
    if 'terrible' in dict:
        return dict['terrible']
    else:
        return 0

In [45]:
products['terrible'] = products['word_count'].apply(terrible_count)

In [46]:
def awful_count(dict):
    if 'awful' in dict:
        return dict['awful']
    else:
        return 0

In [47]:
products['awful'] = products['word_count'].apply(awful_count)

In [48]:
def wow_count(dict):
    if 'wow' in dict:
        return dict['wow']
    else:
        return 0

In [49]:
products['wow'] = products['word_count'].apply(wow_count)

In [50]:
def hate_count(dict):
    if 'hate' in dict:
        return dict['hate']
    else:
        return 0

In [51]:
products['hate'] = products['word_count'].apply(hate_count)

In [52]:
products.head(5)

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	sentiment	int
	awesome	int
	great	int
	fantastic	int
	amazing	int
	love	int
	horrible	int
	bad	int
	terrible	int
	awful	int
	wow	int
	hate	int

Rows: 5

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+-----------+---------+-------+-----------+---------+
|           word_count          | sentiment | awesome | great | fantastic | amazing |
+-------------------------------+-----------+---------+-------+-----------+---------+
| {'and': 3, 'love': 1, 'it'... |     1     |    0    |   0   |     0     |    0    |
| {'and': 2, 'quilt': 1, 'it... |     1     |    0    |   0   |     0     |    0    |
| {'ingenious': 1, 'and': 3,... |     1     |    0    |   0   |     0     |    0    |
| {'and': 2, 'parents!!': 1,... |     1     |    0    |   1   |     0     |    0    |
| {'and': 2, 'this': 2, 'her... |     1     |    0    |   1   |     0     |    0    |
+-------------------------------+-----------+---------+-------+-----------+---------+
+------+----------+-----+----------+-------+-----+------+
| love | horrible | bad | terrible | awful | wow | hate |
+------+----------+-----+----------+-------+-----+------+
|  1   |    0     |  0  |    0     |   0   |  0  |  0   |
|  0   |    0     |  0  |    0     |   0   |  0  |  0   |
|  2   |    0     |  0  |    0     |   0   |  0  |  0   |
|  0   |    0     |  0  |    0     |   0   |  0  |  0   |
|  0   |    0     |  0  |    0     |   0   |  0  |  0   |
+------+----------+-----+----------+-------+-----+------+
[5 rows x 16 columns]

In [53]:
def col_sum(list):
    l = []
    for word in list:
        return products[word].sum()

In [54]:
l = [products['awesome'].sum(),products['great'].sum(),products['fantastic'].sum()]

In [55]:
products['amazing'].sum()

1305

In [56]:
products['love'].sum()

40277

In [57]:
products['horrible'].sum()

659

In [58]:
products['bad'].sum()

3197

In [59]:
products['terrible'].sum()

673

In [60]:
products['awful'].sum()

345

In [61]:
products['wow'].sum()

131

In [62]:
products['hate'].sum()

1057

### Create new model 

In [63]:
train_data,test_data = products.random_split(.8, seed=0)

In [64]:
new_sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                                     target='sentiment',
                                                                     features=selected_words,
                                                                     validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 11
PROGRESS: Number of unpacked features : 11
PROGRESS: Number of coefficients    : 12
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 2        | 0.369121     | 0.844299          | 0.842842            |
PROGRESS: | 2         | 3        | 0.641082     | 0.844186          | 0.842842            |
PROGRESS: | 3         | 4        | 0.962160     | 0.844276          | 0.843142            |
PROGRESS: | 4         | 5        |

In [65]:
new_sentiment_model['coefficients']

name,index,class,value
(intercept),None,1,1.36728315229
awesome,None,1,1.05800888878
great,None,1,0.883937894898
fantastic,None,1,0.891303090304
amazing,None,1,0.892802422508
love,None,1,1.39989834302
horrible,None,1,-1.99651800559
bad,None,1,-0.985827369929
terrible,None,1,-2.09049998487
awful,None,1,-1.76469955631


In [66]:
new_sentiment_model['coefficients'].sort('value', ascending = False)

name,index,class,value
love,None,1,1.39989834302
(intercept),None,1,1.36728315229
awesome,None,1,1.05800888878
amazing,None,1,0.892802422508
fantastic,None,1,0.891303090304
great,None,1,0.883937894898
wow,None,1,-0.0541450123333
bad,None,1,-0.985827369929
hate,None,1,-1.40916406276
awful,None,1,-1.76469955631


### 3. Comparing the accuracy of different sentiment analysis model

In [67]:
sentiment_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +------------------+----------------+------------------+-------+------+
 |    threshold     |      fpr       |       tpr        |   p   |  n   |
 +------------------+----------------+------------------+-------+------+
 |       0.0        | 0.219252336449 | 0.00489512988173 | 27987 | 5350 |
 | 0.0010000000475  | 0.780747663551 |  0.995104870118  | 27987 | 5350 |
 | 0.00200000009499 | 0.74261682243  |  0.993604173366  | 27987 | 5350 |
 | 0.00300000002608 | 0.720560747664 |  0.99278236324   | 27987 | 5350 |
 | 0.00400000018999 | 0.705420560748 |  0.992210669239  | 27987 | 5350 |
 | 0.00499999988824 | 0.694205607477 |  0.991817629614  | 27987 | 5350 |
 | 0.00600000005215 | 0.684485981308 |  0.991281666488  | 27987 | 5350 |
 | 0.00700000021607 | 0.674579439252 |  0.990781434237  | 27987 | 5350 |
 | 0.00800000037998 | 0.664485981308 |  0.990459856362  | 27987 | 5350 |
 | 0.00899999961257 

In [68]:
new_sentiment_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +------------------+-------------------+-----+-------+------+
 |    threshold     |        fpr        | tpr |   p   |  n   |
 +------------------+-------------------+-----+-------+------+
 |       0.0        | 0.000187617260788 | 0.0 | 27995 | 5330 |
 | 0.0010000000475  |   0.999812382739  | 1.0 | 27995 | 5330 |
 | 0.00200000009499 |   0.999624765478  | 1.0 | 27995 | 5330 |
 | 0.00300000002608 |   0.999624765478  | 1.0 | 27995 | 5330 |
 | 0.00400000018999 |   0.999437148218  | 1.0 | 27995 | 5330 |
 | 0.00499999988824 |   0.999437148218  | 1.0 | 27995 | 5330 |
 | 0.00600000005215 |   0.999249530957  | 1.0 | 27995 | 5330 |
 | 0.00700000021607 |   0.999249530957  | 1.0 | 27995 | 5330 |
 | 0.00800000037998 |   0.999249530957  | 1.0 | 27995 | 5330 |
 | 0.00899999961257 |   0.999249530957  | 1.0 | 27995 | 5330 |
 +------------------+-------------------+-----+-------+------+
 [1001 rows x 

In [69]:
sentiment_model.show(view='Evaluation')

In [70]:
new_sentiment_model.show(view='Evaluation')

### investigate 

In [71]:
diaper_champ_reviews = products[products['name'] == 'Baby Trend Diaper Champ']

In [72]:
diaper_champ_reviews['predicted_sentiment'] = new_sentiment_model.predict(diaper_champ_reviews, output_type='probability')

In [77]:
diaper_champ_reviews.head()

name,review,rating,word_count,sentiment,awesome
Baby Trend Diaper Champ,I LOVE LOVE LOVE thisproduct! It is SO much ...,4.0,"{'rating': 1,'contacted': 1, 'over': ...",1,0
Baby Trend Diaper Champ,I received my DiaperChamp at my baby shower ...,5.0,"{'bags.': 1, ""don't"": 1,'son.': 1, 'of,': 1, ...",1,0
Baby Trend Diaper Champ,"Love it, love it, loveit! This lives up to ...",5.0,"{'instead': 1, 'all': 1,'already': 1, 'love': 3, ...",1,0
Baby Trend Diaper Champ,Works great - no smells.LOVE that it uses reg ...,5.0,"{'and': 2, 'bags.': 1,'garbage': 1, 'wastef ...",1,0
Baby Trend Diaper Champ,I love this diaper paleand wouldn't dream of ...,5.0,"{'and': 3, 'love': 1,'use.': 1, 'is': 2, ' ...",1,0
Baby Trend Diaper Champ,I've worked with kidsmore than half my life. ...,5.0,"{'and': 4, 'genies': 1,'all': 1, 'because': 1, ...",1,0
Baby Trend Diaper Champ,I love this diaper pail.It keeps the diapers ...,4.0,"{'and': 1, 'old': 1,'extra': 1, 'is': 1, ...",1,0
Baby Trend Diaper Champ,"This is absolutely, byfar, the best diaper ...",5.0,"{'just': 3, 'money': 1,'still': 3, 'fine': 1, ...",1,0
Baby Trend Diaper Champ,Love the Diaper Champ. Ihad planned to get the ...,4.0,"{'reviews,': 1, 'infant':1, 'bags.': 1, 'just' ...",1,0
Baby Trend Diaper Champ,We had 2 diaper Genie'sboth given to us as a ...,4.0,"{'hand.': 1, 'both': 1,'(required': 1, 'befo ...",1,0


In [76]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

In [78]:
new_sentiment_model.predict(diaper_champ_reviews[0:1], output_type='probability')

dtype: float
Rows: 1
[0.9984234145936198]

In [79]:
diaper_champ_reviews[0]['review']

'I LOVE LOVE LOVE this product! It is SO much easier to use than the Diaper Genie, (you need a PHD in poopy to figure out how to use the darn thing!) and it even takes the same bags as my kitchen trash can, shich is super convenient, and cost efficient as I can buy them in bulk.The only reason for not rating it a 5 star was that I did have one small problem with it. The foam gasket in the barrell which keeps the poopy smell inside the unit ripped somehow, and it got VERY stinky. HOWEVER, I contacted the manufacturer though their website, and received an email back the same day stating that this was unusual, and that replacement gaskets were on their way to me. They arrived inside of a week and after replacing, it works great again! (They even sent me extras should it happen again)I HIGHLY reccomend this diaper pail over ANY competitors, you will not be sorry!'

In [80]:
diaper_champ_reviews[0]['word_count']

{'(they': 1,
 '(you': 1,
 '5': 1,
 'a': 3,
 'after': 1,
 'again!': 1,
 'again)i': 1,
 'an': 1,
 'and': 6,
 'any': 1,
 'arrived': 1,
 'as': 2,
 'back': 1,
 'bags': 1,
 'barrell': 1,
 'be': 1,
 'bulk.the': 1,
 'buy': 1,
 'can': 1,
 'can,': 1,
 'competitors,': 1,
 'contacted': 1,
 'convenient,': 1,
 'cost': 1,
 'darn': 1,
 'day': 1,
 'diaper': 2,
 'did': 1,
 'easier': 1,
 'efficient': 1,
 'email': 1,
 'even': 2,
 'extras': 1,
 'figure': 1,
 'foam': 1,
 'for': 1,
 'gasket': 1,
 'gaskets': 1,
 'genie,': 1,
 'got': 1,
 'great': 1,
 'happen': 1,
 'have': 1,
 'highly': 1,
 'how': 1,
 'however,': 1,
 'i': 4,
 'in': 3,
 'inside': 2,
 'is': 2,
 'it': 6,
 'it.': 1,
 'keeps': 1,
 'kitchen': 1,
 'love': 3,
 'manufacturer': 1,
 'me': 1,
 'me.': 1,
 'much': 1,
 'my': 1,
 'need': 1,
 'not': 2,
 'of': 1,
 'on': 1,
 'one': 1,
 'only': 1,
 'out': 1,
 'over': 1,
 'pail': 1,
 'phd': 1,
 'poopy': 2,
 'problem': 1,
 'product!': 1,
 'rating': 1,
 'reason': 1,
 'reccomend': 1,
 'received': 1,
 'replacement': 1,

In [83]:
diaper_champ_reviews.head(1)

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	sentiment	int
	awesome	int
	great	int
	fantastic	int
	amazing	int
	love	int
	horrible	int
	bad	int
	terrible	int
	awful	int
	wow	int
	hate	int
	predicted_sentiment	float

Rows: 1

Data:
+-------------------------+-------------------------------+--------+
|           name          |             review            | rating |
+-------------------------+-------------------------------+--------+
| Baby Trend Diaper Champ | I LOVE LOVE LOVE this prod... |  4.0   |
+-------------------------+-------------------------------+--------+
+-------------------------------+-----------+---------+-------+-----------+---------+
|           word_count          | sentiment | awesome | great | fantastic | amazing |
+-------------------------------+-----------+---------+-------+-----------+---------+
| {'rating': 1, 'contacted':... |     1     |    0    |   1   |     0     |    0    |
+-------------------------------+-----------+---------+-------+-----------+---------+
+------+----------+-----+----------+-------+-----+------+---------------------+
| love | horrible | bad | terrible | awful | wow | hate | predicted_sentiment |
+------+----------+-----+----------+-------+-----+------+---------------------+
|  3   |    0     |  0  |    0     |   0   |  0  |  0   |    0.998423414594   |
+------+----------+-----+----------+-------+-----+------+---------------------+
[1 rows x 17 columns]